In [ ]:
import imageio
import pickle
from PIL import Image
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
from utils import *

In [ ]:
#Get classes
CLASS_URL = 'https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl'
class_dict = pickle.load(urllib.request.urlopen(CLASS_URL))

#Retrieve image and assign class
IMG_URL = "https://bloximages.chicago2.vip.townnews.com/chippewa.com/content/tncms/assets/v3/editorial/c/ca/cca0d87d-8d7c-520f-b012-bc065d18b3dd/553cbf5aa0ffb.image.jpg?crop=200%2C200%2C0%2C8&resize=1200%2C1200&order=crop%2Cresize"
#IMG_URL = "https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSa3p29cIlgj0yQwMuLZzh8PKkgBUzbPLWrU-7K79DjDL498JsA&usqp=CAU"
#IMG_URL = "https://thumbs-prod.si-cdn.com/D9GA8QHJQ70nFNQB-AFZ41S-Bs0=/fit-in/1600x0/https://public-media.si-cdn.com/filer/ea/21/ea2159df-9eec-4b05-a3de-356c57e23227/another_airplane_4676723312.jpg"
#IMG_URL = "https://5.imimg.com/data5/OX/QV/SF/SELLER-23061093/national-35-model-air-rifle-500x500.jpg"
classidx = 919
#classidx = 36
response = requests.get(IMG_URL)
img = Image.open(io.BytesIO(response.content))
print(img.size)
imshow(img)

In [ ]:
#Model parameters
model_img_size = 224
model_transform = transforms.Compose([transforms.Resize((model_img_size,model_img_size)),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])])
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)
forward_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

print(type(img))
img_input = model_transform(img)
#img_input = img_input.unsqueeze(0)
#img_input = Variable(img_input)
img_input.shape

In [ ]:
#Model definition
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
pretrained_model = models.resnet101(pretrained=True)
pretrained_model.to(device)
pretrained_model.eval()

#Pass input through model
img_input = img_input.to(device)
pred = pretrained_model(img_input.unsqueeze(0))

#Use softmax to get predicted probability and view it
lay2 = torch.nn.Softmax(dim=1)
prob = lay2(pred)
maxOcls = prob.max(1)
print("Class is {} ({}) with confidence {}%".format(maxOcls.indices.item(),class_dict[maxOcls.indices.item()],maxOcls.values.item()*100))

In [ ]:
def get_object_mask(input_image):
    #print(test[-3][:])
    test = inv_normalize(input_image)
    test = np.uint8(test.numpy()*255).transpose((1,2,0))
    test = 1 - test
    #test = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(test, cv2.COLOR_RGB2GRAY)
    th, thgray = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY_INV); 
    #blurred = cv2.GaussianBlur(thgray, (9, 9), 0)
    th, blurred = cv2.threshold(thgray, 70, 255, cv2.THRESH_BINARY)
    edged = cv2.Canny(blurred, 1, 250, L2gradient=True)   
    #imgplot = plt.imshow(blurred, cmap="gray")
    #plt.show()
    #imshow(input_image[0])
    #applying closing function
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))
    closed = cv2.morphologyEx(blurred, cv2.MORPH_CLOSE, kernel)    #finding_contours
    (cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)   
    #cnts = [] #only for plate class 
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        cv2.drawContours(closed, [approx], -1, (0, 255, 0), 2)    
    th, im_th = cv2.threshold(closed, 254, 255, cv2.THRESH_BINARY_INV);    
    h, w = im_th.shape[:2]
    im_floodfill = im_th.copy()
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_floodfill, mask, (0,0), 0);    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)    # Combine the two images to get the foreground.
    #imshow(im_th)
    #plt.show()
    #imshow(im_floodfill_inv)
    #plt.show()
    im_out = im_th ^ im_floodfill_inv
    im_out = cv2.GaussianBlur(im_out, (3, 3), 0)
    return im_out
print(type(img_input))
imshow_tensor(img_input.cpu(),inv_normalize)
#imm = Image.fromarray(get_object_mask(img_input.cpu()))
#imm.save("class919_mask.png")
imgplot = plt.imshow(get_object_mask(img_input.cpu()))
plt.show()

In [ ]:
mask = torch.tensor(get_object_mask(img_input.cpu()), dtype=torch.float, device=device)
mask = mask / torch.max(mask)

In [ ]:
def applyMask(w, batch_limits, mask, c_limits):
    
    sz = w.shape[1]
    
    #stack the signal to fit the input size
    oot = stack(w,228)             
    batch = batch_limits[1]-batch_limits[0]
    # EOT sampling for ambient light and shift
    c = torch.rand([batch,1,1,1], device=device) * (c_limits[1] - c_limits[0]) + c_limits[0]
    shift = torch.tensor(range(batch_limits[0],batch_limits[1]), dtype=torch.int)
    #shift = torch.from_numpy(np.array(range(0,batch,1)))
    #Shift the signal
    ootn = shift_operation(oot.unsqueeze(0).repeat(batch,1,1,1).view(-1, 228, 1), shift).view(batch,3,228,1)
    #Fit w into the range [0,1]. new_w is the same as ft
    new_w = .5 * (torch.tanh(ootn) + 1)
    
    #Convolution of ft and the shutter
    gy = lay(new_w.transpose(0,3).transpose(0,1)).transpose(0,1).transpose(0,3)
    #Mask the signal to only affect the object
    gy_mask = gy * mask
    #return 0.5*c, new_w, c
    return (c+ (1-c)*gy_mask), new_w, c

In [ ]:
def getPrediction(img, classid):
    img_input = model_transform(img)
    img_input = img_input.to(device)
    pred = pretrained_model(img_input.unsqueeze(0))

    #Use softmax to get predicted probability and view it
    lay2 = torch.nn.Softmax(dim=1)
    prob = lay2(pred)
    maxOcls = prob.max(1)
    return class_dict[maxOcls.indices.item()], round(maxOcls.values.item()*100, 2)

In [ ]:
def getAdvPrediction(img, classid):
    img_input = model_transform(img)
    img_input = img_input.to(device)
    w = torch.load('w_3_channel.pt')
    w = w.to(device)
    out_prediction = []
    out_imgs = []
    for j in range(0,w.shape[1],64):
        batch_limits = [j, min(w.shape[1],j+64)]

        gy, signal, c = applyMask(w,batch_limits, mask, [0.4,0.7])
        inp = gy*inv_normalize(img_input)   
        inp = torch.cat([forward_normalize(i).unsqueeze(0) for i in inp])
        out = pretrained_model(inp)
        prob = lay2(out)
        #imshow_tensor(inp[1].detach().cpu(),inv_normalize)
        tran = transforms.ToPILImage()
        maxcls = prob.max(1)
        #print(out.topk(2).indices[0])
        
        for k in range(batch_limits[1]-batch_limits[0]):
            #print("alpha: {}".format(c[i].item()))
            c_value = round(c[k][0][0][0].data.item(),4)
            out_imgs.append(tran(inv_normalize(inp[k].detach().cpu())))
            out_prediction.append((class_dict[maxcls.indices[k].item()], round(maxcls.values[k].item()*100,4),class_dict[classid],round(prob[k][classid].data.item()*100,4),c_value))
        del out
        del inp
        del signal
        del gy
        del prob
        del maxcls
    return out_prediction, out_imgs

In [ ]:
out_prediction, out_img = getAdvPrediction(img, 919)
#while(len([o for o in out_prediction if o[0]!="street sign"]) < 110):
#    out_prediction, out_img = getAdvPrediction(img, 919)
#    print(len([o for o in out_prediction if o[0]!="street sign"]))
out_prediction
vals = np.array([o[3] for o in out_prediction if o[2]==class_dict[919]])
vals.mean()

In [ ]:
img_ann_tm = annonate(border(out_img[0]), out_prediction[0])
display(img_ann_tm)
img_ann_tm.save("animation_images/919_0.5.png")

In [ ]:
def writeToGif(filenames,fpss):
    with imageio.get_writer('animation_images/movie.gif', mode='I',fps=fpss) as writer:
        for image in filenames:
            #image = imageio.imread(filename)
            writer.append_data(image)

In [ ]:
def border(old_im):
    #old_im = Image.open('c0.7_sz46_tg861.png')
    old_size = old_im.size
    new_size = (230, 260)
    new_im = Image.new("RGB", new_size,"#FFF")   ## luckily, this is already black!
    new_im.paste(old_im, (3,0))

    #display(new_im)
    return new_im
# new_im.save('someimage.jpg')

In [ ]:
def annonate(img, out):
    draw = ImageDraw.Draw(img)
    # font = ImageFont.truetype(<font-file>, <font-size>)
    #font = ImageFont.load_default()
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 10, encoding="unic")
    # draw.text((x, y),"Sample Text",(r,g,b))
    draw.text((0, 224),"Ambient Light = "+str(out[4]),(0,0,0),font=font)
    draw.text((0, 235),out[0][:15]+" = "+str(out[1]),(0,0,0),font=font)
    draw.text((0, 246),out[2]+" = "+str(out[3]),(0,0,0),font=font)
    #display(img)
    return img
#img.save('sample-out.jpg')

In [ ]:
annonated_imgs = []
for i in range(0,len(out_img),5):
    im = out_img[i]
    #if out_prediction[i][0]=="street sign": continue
    img_ann = annonate(border(im), out_prediction[i])
    #filenames.append("animation_images/shift_"+str(i)+".jpg")
    annonated_imgs.append(np.array(img_ann))
    #img.save("animation_images/shift"+str(i)+".jpg")
writeToGif(annonated_imgs, 5)